In [0]:
from keras import Model, optimizers
from keras.applications import ResNet50
from keras.layers import Dense, Flatten, Dropout
from keras_preprocessing.image import ImageDataGenerator

import pandas as pd
from PIL import Image
import numpy as np

class ImageProcessingModel:
    VALIDATION_SPLIT = 0.1
    RESNET_TRANSFERRED_LAYER_COUNT = 5

    @staticmethod
    def load_image(image_path):
        img = Image.open(image_path)
        img.load()
        data = np.asarray(img, dtype="int32")
        return data

    def __init__(self, dataset_directory_name, metadata_file_name, x_col, y_col, target_size):
        dataset = pd.read_csv(f"{dataset_directory_name}/{metadata_file_name}")

        dataset["image_id"] = dataset_directory_name + "/" + dataset[x_col] + ".jpg"

        total_image_count = len(dataset)
        training_image_count = total_image_count * (1 - ImageProcessingModel.VALIDATION_SPLIT)
        validation_image_count = total_image_count - training_image_count

        image_data_generator = ImageDataGenerator(validation_split = ImageProcessingModel.VALIDATION_SPLIT)

        batch_size = 1

        self.train_image_data_yielder = image_data_generator.flow_from_dataframe(dataframe = dataset, batch_size = batch_size, x_col = x_col, y_col = y_col, target_size = target_size, subset = "training")
        self.validate_image_data_yielder = image_data_generator.flow_from_dataframe(dataframe = dataset, batch_size = batch_size, x_col = x_col, y_col = y_col, target_size = target_size, subset = "validation")

        transferred_model = ResNet50(include_top = False, weights = "imagenet", input_shape = (*target_size, 3))

        transferred_model.load_weights("resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

        for frozen_layer in transferred_model.layers[:ImageProcessingModel.RESNET_TRANSFERRED_LAYER_COUNT]:
            frozen_layer.trainable = False

        extra_layer = transferred_model.output
        extra_layer = Flatten()(extra_layer)
        extra_layer = Dropout(0.5)(extra_layer)

        class_count = dataset[y_col].nunique()
        predictions = Dense(class_count, activation = "softmax")(extra_layer)

        self.final_model = Model(input = transferred_model.input, output = predictions)
        final_model_optimizer = optimizers.SGD(lr = 10e-2)

        self.final_model.compile(loss = "categorical_crossentropy", optimizer = final_model_optimizer, metrics = ["accuracy"])

        self.training_steps_per_epoch = training_image_count // batch_size
        self.validation_steps_per_epoch = validation_image_count // batch_size

    def train(self, epochs):
        self.final_model.fit_generator(self.train_image_data_yielder, steps_per_epoch = self.training_steps_per_epoch, epochs = epochs, validation_data = self.validate_image_data_yielder, validation_steps = self.validation_steps_per_epoch)

class ISICModel(ImageProcessingModel):
    def __init__(self):
        super().__init__("isic", "HAM10000_metadata.csv", "image_id", "dx", (600, 450))